## Deliverable 2. Create a Customer Travel Destinations Map.

In [76]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from configG import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [77]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Poum,NC,-20.2333,164.0167,70.56,67,100,5.99,overcast clouds
1,1,Marzuq,YE,14.4000,46.4667,84.74,34,89,6.58,overcast clouds
2,2,Ostrovnoy,RU,68.0531,39.5131,54.27,79,5,9.55,clear sky
3,3,Yellowknife,CA,62.4560,-114.3525,59.52,67,90,25.32,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,38.48,89,100,1.39,overcast clouds


In [78]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [79]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Marzuq,YE,14.4000,46.4667,84.74,34,89,6.58,overcast clouds
7,7,Hilo,US,19.7297,-155.0900,82.26,80,68,3.00,broken clouds
10,10,Kirakira,SB,-10.4544,161.9205,84.40,67,96,6.51,overcast clouds
15,15,Patti,IN,31.2808,74.8572,80.55,73,5,5.55,clear sky
22,22,Gat,IL,31.6100,34.7642,81.79,100,8,6.08,clear sky
26,26,Georgetown,MY,5.4112,100.3354,81.90,87,20,2.30,thunderstorm
27,27,Albany,US,42.6001,-73.9662,77.52,84,100,3.20,overcast clouds
28,28,Butaritari,KI,3.0707,172.7902,82.81,65,13,14.09,few clouds
32,32,Jamestown,US,42.0970,-79.2353,75.36,88,100,2.89,overcast clouds
33,33,Coari,BR,-4.0850,-63.1414,78.91,65,59,3.51,broken clouds


In [80]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                255
City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [83]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# clean_df= preferred_cities_df.replace(" ", np.nan, inplace=True)
clean_df= preferred_cities_df.dropna()

In [84]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Marzuq,YE,84.74,overcast clouds,14.4000,46.4667,
7,Hilo,US,82.26,broken clouds,19.7297,-155.0900,
10,Kirakira,SB,84.40,overcast clouds,-10.4544,161.9205,
15,Patti,IN,80.55,clear sky,31.2808,74.8572,
22,Gat,IL,81.79,clear sky,31.6100,34.7642,
26,Georgetown,MY,81.90,thunderstorm,5.4112,100.3354,
27,Albany,US,77.52,overcast clouds,42.6001,-73.9662,
28,Butaritari,KI,82.81,few clouds,3.0707,172.7902,
32,Jamestown,US,75.36,overcast clouds,42.0970,-79.2353,
33,Coari,BR,78.91,broken clouds,-4.0850,-63.1414,


In [85]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    hotel_df.head()     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [86]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Hilo,US,82.26,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Kirakira,SB,84.40,overcast clouds,-10.4544,161.9205,SanBiz Lodge
15,Patti,IN,80.55,clear sky,31.2808,74.8572,SOKHI NIWAS
22,Gat,IL,81.79,clear sky,31.6100,34.7642,OlusHome
26,Georgetown,MY,81.90,thunderstorm,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
663,Jiazi,CN,83.10,overcast clouds,22.8779,116.0670,Hotel Jin Hao
664,Thessalon,CA,75.69,broken clouds,46.2501,-83.5666,Carolyn Beach Inn & Restaurant
666,Guerrero Negro,MX,77.41,scattered clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
667,Pascagoula,US,87.85,clear sky,30.3658,-88.5561,Regency Inn


In [87]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="Hotel_ID")

In [88]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [89]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))